看上博客上一个作者的文章，想一次性下载到一个word文件中，并且可以设置好目录，通过word的“导航窗格”快速定位单篇文章。一劳永逸，从此再也不用去博客上一篇一篇地翻阅了。整理一下步骤：
1. 先获取到所有文章的标题、发表日期、链接
2. 通过链接获取文章的内容
3. 将文章标题作为“1级”，发表日期和内容作为正文写入word文件
4. 保存wrod文件
<br/>
<br/>
下面就按照以上步骤进行操作。

先进入到目标博客的主页，点击“博文目录”，这样就在网址栏看到“blog.sina.com.cn/s/articlelist_5119330124_0_1.html” 。再点击下一页，可以看到网址末尾的“1”变成了“2”。这样我们就知道所有页对应的网址了（尾号从1到5）。

![](images\blog_next.png)

先挑第一页的网址，定位我们需要的信息，以便后续批量爬取。在博文的标题和发表日期上分别点右键，选择“检查”，在浏览器右侧看到如下信息。可见博文标题和博文链接都位于`class="atc_title"`下面，发表时间位于`class="atc_tm SG_txtc"`下面。因此使用`soup.select('.atc_title')`就可以获取当前网页的所有博文的链接和标题；使用`soup.select('.atc_tm')`可获取所有博文的发表日期。慢着，不是发表时间对应的`class`是`"atc_tm SG_txtc"`吗？怎么这里只取了`atc_tm`呢？这是因为`atc_tm`和`SG_txtc`之间有个空格，如果原样取，反而找不到目标。取二者之一或者将空格换成"."才能定位目标。这个涉及到网页编程语言了，就不深究了，我们只需要知道有这个坑，并且了解如何避坑就行。

![](images\title.png)
![](images\date.png)

In [30]:
import requests
from bs4 import BeautifulSoup
url = f'http://blog.sina.com.cn/s/articlelist_5119330124_0_1.html'
wb_data = requests.get(url)
soup = BeautifulSoup(wb_data.content)

In [31]:
#获取当页所有文章的标题和链接
soup.select('.atc_title')

[<span class="atc_title">
 <a href="http://blog.sina.com.cn/s/blog_13122c74c0102zdsy.html" target="_blank" title="">投资难在慢成毁于速成</a></span>,
 <span class="atc_title">
 <a href="http://blog.sina.com.cn/s/blog_13122c74c0102zd8r.html" target="_blank" title="">投机性博弈</a></span>,
 <span class="atc_title">
 <a href="http://blog.sina.com.cn/s/blog_13122c74c0102zd7m.html" target="_blank" title="">薅羊毛操作：兖州煤业股份</a></span>,
 <span class="atc_title">
 <a href="http://blog.sina.com.cn/s/blog_13122c74c0102zd75.html" target="_blank" title="">指数增长的重要配方：极低的边际成…</a></span>,
 <span class="atc_title">
 <a href="http://blog.sina.com.cn/s/blog_13122c74c0102zd4s.html" target="_blank" title="">价值回归套利体系目录</a></span>,
 <span class="atc_title">
 <a href="http://blog.sina.com.cn/s/blog_13122c74c0102zcw5.html" target="_blank" title="">白云机场50%获利减仓：白云山波段…</a></span>,
 <span class="atc_title">
 <a href="http://blog.sina.com.cn/s/blog_13122c74c0102zcvj.html" target="_blank" title="">企业价值是动态的：套利却依赖于静…</a></span>,
 <span c

In [32]:
#获取当页所有文章的发表时间
soup.select('.atc_tm')

[<span class="atc_tm SG_txtc">2019-09-17 21:03</span>,
 <span class="atc_tm SG_txtc">2019-09-03 14:01</span>,
 <span class="atc_tm SG_txtc">2019-09-02 20:01</span>,
 <span class="atc_tm SG_txtc">2019-09-02 08:38</span>,
 <span class="atc_tm SG_txtc">2019-08-29 21:54</span>,
 <span class="atc_tm SG_txtc">2019-08-22 10:56</span>,
 <span class="atc_tm SG_txtc">2019-08-21 20:09</span>,
 <span class="atc_tm SG_txtc">2019-08-20 20:48</span>,
 <span class="atc_tm SG_txtc">2019-08-19 13:07</span>,
 <span class="atc_tm SG_txtc">2019-08-15 14:01</span>,
 <span class="atc_tm SG_txtc">2019-07-23 16:23</span>,
 <span class="atc_tm SG_txtc">2019-07-22 16:24</span>,
 <span class="atc_tm SG_txtc">2019-07-18 16:03</span>,
 <span class="atc_tm SG_txtc">2019-06-28 15:41</span>,
 <span class="atc_tm SG_txtc">2019-06-18 17:16</span>,
 <span class="atc_tm SG_txtc">2019-06-12 14:28</span>,
 <span class="atc_tm SG_txtc">2019-06-11 09:00</span>,
 <span class="atc_tm SG_txtc">2019-06-03 14:08</span>,
 <span cla

如上获取的文章标题及链接信息是存在一个大列表中的。现在以第一个元素为例从中提取出链接和标题信息。观察发现链接位于`a`标签里的`href`里面，于是使用`select`方法选中`a`标签，可以看到结果是一个新的列表（如下）。

In [34]:
soup.select('.atc_title')[0].select('a')

[<a href="http://blog.sina.com.cn/s/blog_13122c74c0102zdsy.html" target="_blank" title="">投资难在慢成毁于速成</a>]

然后再从这个新列表中提取出链接和标题。使用`get("href")`方法获得链接；使用`text`方法获得标题。

In [36]:
soup.select('.atc_title')[0].select('a')[0].get("href")

'http://blog.sina.com.cn/s/blog_13122c74c0102zdsy.html'

In [37]:
soup.select('.atc_title')[0].select('a')[0].text

'投资难在慢成毁于速成'

发表时间的获取就简单很多了，直接用`text`方法即可。

In [38]:
soup.select('.atc_tm')[0].text

'2019-09-17 21:03'

单页的信息搞定，然后就可以批量处理了。使用`for`循环遍历所有页，然后逐个提取。因为我们已知作者的文章共有5页，所以直接使用`range(1,6)`。将最终的信息存入字典`all_links`。其中，“标题”作为键，文章链接和发表时间作为值。通过`len(all_links)`查看获取的文章链接数，一共211篇文章。

In [1]:
#获取所有博客文章的链接
import requests
from bs4 import BeautifulSoup

all_links = {}
for i in range(1,6):
    url = f'http://blog.sina.com.cn/s/articlelist_5119330124_0_{i}.html'
    wb_data = requests.get(url)
    soup = BeautifulSoup(wb_data.content)
    links = soup.select('.atc_title')
    times = soup.select('.atc_tm')
    for i in range(len(links)):
        http_link = links[i].select('a')[0].get('href')
        title = links[i].text.strip()
        time = times[i].text
        all_links[title] = [http_link, time] 

In [2]:
len(all_links)

211

In [3]:
all_links

{'投资难在慢成毁于速成': ['http://blog.sina.com.cn/s/blog_13122c74c0102zdsy.html',
  '2019-09-17 21:03'],
 '投机性博弈': ['http://blog.sina.com.cn/s/blog_13122c74c0102zd8r.html',
  '2019-09-03 14:01'],
 '薅羊毛操作：兖州煤业股份': ['http://blog.sina.com.cn/s/blog_13122c74c0102zd7m.html',
  '2019-09-02 20:01'],
 '指数增长的重要配方：极低的边际成…': ['http://blog.sina.com.cn/s/blog_13122c74c0102zd75.html',
  '2019-09-02 08:38'],
 '价值回归套利体系目录': ['http://blog.sina.com.cn/s/blog_13122c74c0102zd4s.html',
  '2019-08-29 21:54'],
 '白云机场50%获利减仓：白云山波段…': ['http://blog.sina.com.cn/s/blog_13122c74c0102zcw5.html',
  '2019-08-22 10:56'],
 '企业价值是动态的：套利却依赖于静…': ['http://blog.sina.com.cn/s/blog_13122c74c0102zcvj.html',
  '2019-08-21 20:09'],
 '价值套利之以自身优势构建能力圈': ['http://blog.sina.com.cn/s/blog_13122c74c0102zcuu.html',
  '2019-08-20 20:48'],
 '长电继续穿越牛熊': ['http://blog.sina.com.cn/s/blog_13122c74c0102zcte.html',
  '2019-08-19 13:07'],
 '为什么要调研': ['http://blog.sina.com.cn/s/blog_13122c74c0102zcpm.html',
  '2019-08-15 14:01'],
 '化妆品调研随笔': ['http://b

拿到所有文章链接后，先取一个来测试一下如何获取页面的文字。在文字上点右键，选择“检查”，可见其内容位于`class=articalContent   newfont_family`里面，因此使用`soup.select(".articalContent.newfont_family")`就可以获取到（注意articalContent和newfont_family之间的空格要用"."代替）。将其存入`article`变量，显示一下，可以看到这是一个大列表，其中的文本就是我们需要的内容。下面就需要将文本单独提取出来。

![](images\article.png)

In [40]:
#获取单篇文章中的文字
url = 'http://blog.sina.com.cn/s/blog_13122c74c0102zbt3.html'
wb_data = requests.get(url)
soup = BeautifulSoup(wb_data.content)
article = soup.select(".articalContent.newfont_family")
article

[<div class="articalContent newfont_family" id="sina_keyword_ad_area2">
 <font color="#FF0000" face="楷体_GB2312, 楷体" style="font-size: 16px;"> <wbr>
 今日科创板首日上市，可谓风光大嫁，俗话说春宵一刻值千金，姑娘还是那个姑娘，只是今日是最值钱的一刻，更有甚者，拿配资杠杆账户去买这姑娘，实属愚蠢心急至极，我想说真正的好姑娘未必是那个嫁妆最贵的，除掉镁光灯，我们依然会看到丑陋的素颜，普通夜场的姑娘卖出了天上人间的价格，脱离了这天上人间，以后照样是包夜几百，这就是市场情绪，朦胧美也是一种美，但却不是经得起时间真正的美，对此，我只好躲在角落里静静感受下市场以外疯狂的人心，暴烈的人性，科创板集体发疯无异于婊子集体宣誓涨价，皮条客圆满完成任务，可回家烫壶好酒，剩下的只是市场的一地鸡毛，历来如此，科创板也定然逃不过，散户看看罢了，切莫当真。</wbr></font>
 <div><font color="#FF0000" face="楷体_GB2312, 楷体" style="font-size: 16px;"> <wbr>
 有人说明明知道很多却为何也亏损，以个人经验，投资有几重境界，关键要看跟自己的认知圈能否完全匹配，有人只知道公司会赚钱，却不会知道公司能否赚更多的钱，亦或是只知道公司的模式很好，但却也不知这种模式能否盈利更强的利润，总之，二级市场投资是一个踩高跷的游戏，仅知道公司赚钱是远远不够的，它考验我们的是要知道公司为何而增长以及确定公司会增长，知道增长跟知道公司会高速增长还是两回事，故，这是二级市场投资中最为艰难的，说白了，它涉及到预测未来，以及对某个行业的直观智慧，至于某些分析，看看就好，同样一句话，经历过的说出来，对于那些没有经历过的人而言，是永远也无法听懂的，能理解的也只是字面意思，还得回归那句老话，纸上得来终觉浅，绝知此事要躬行。这也是个人几十年如一日不断调研游走于底层市场的原因，看书之后你会觉得什么都懂，当你游走底层之后，你会觉得什么都不懂，很多问题都是书上没有答案的，都必须由自己提出一种不同的理论角度来解释。</wbr></font></div>
 <div><font color="#FF0000" face=

直接使用`text`方法就能提取出来。“\xa0”是个什么鬼？明显不是我们要的，百度了一下，说是什么不间断空格符。管他呢，直接使用`replace("\xa0","")`删掉，这下就美丽了。“\n”是换行，就不要删了，保持原格式比较好。

In [41]:
article[0].text

'\n\xa0\n今日科创板首日上市，可谓风光大嫁，俗话说春宵一刻值千金，姑娘还是那个姑娘，只是今日是最值钱的一刻，更有甚者，拿配资杠杆账户去买这姑娘，实属愚蠢心急至极，我想说真正的好姑娘未必是那个嫁妆最贵的，除掉镁光灯，我们依然会看到丑陋的素颜，普通夜场的姑娘卖出了天上人间的价格，脱离了这天上人间，以后照样是包夜几百，这就是市场情绪，朦胧美也是一种美，但却不是经得起时间真正的美，对此，我只好躲在角落里静静感受下市场以外疯狂的人心，暴烈的人性，科创板集体发疯无异于婊子集体宣誓涨价，皮条客圆满完成任务，可回家烫壶好酒，剩下的只是市场的一地鸡毛，历来如此，科创板也定然逃不过，散户看看罢了，切莫当真。\n\xa0\n有人说明明知道很多却为何也亏损，以个人经验，投资有几重境界，关键要看跟自己的认知圈能否完全匹配，有人只知道公司会赚钱，却不会知道公司能否赚更多的钱，亦或是只知道公司的模式很好，但却也不知这种模式能否盈利更强的利润，总之，二级市场投资是一个踩高跷的游戏，仅知道公司赚钱是远远不够的，它考验我们的是要知道公司为何而增长以及确定公司会增长，知道增长跟知道公司会高速增长还是两回事，故，这是二级市场投资中最为艰难的，说白了，它涉及到预测未来，以及对某个行业的直观智慧，至于某些分析，看看就好，同样一句话，经历过的说出来，对于那些没有经历过的人而言，是永远也无法听懂的，能理解的也只是字面意思，还得回归那句老话，纸上得来终觉浅，绝知此事要躬行。这也是个人几十年如一日不断调研游走于底层市场的原因，看书之后你会觉得什么都懂，当你游走底层之后，你会觉得什么都不懂，很多问题都是书上没有答案的，都必须由自己提出一种不同的理论角度来解释。\n七雄圣钻继续强势不减，近期继续调研，另，录制价值套利课程中。\n\n\n\n'

In [42]:
article[0].text.replace("\xa0","")

'\n\n今日科创板首日上市，可谓风光大嫁，俗话说春宵一刻值千金，姑娘还是那个姑娘，只是今日是最值钱的一刻，更有甚者，拿配资杠杆账户去买这姑娘，实属愚蠢心急至极，我想说真正的好姑娘未必是那个嫁妆最贵的，除掉镁光灯，我们依然会看到丑陋的素颜，普通夜场的姑娘卖出了天上人间的价格，脱离了这天上人间，以后照样是包夜几百，这就是市场情绪，朦胧美也是一种美，但却不是经得起时间真正的美，对此，我只好躲在角落里静静感受下市场以外疯狂的人心，暴烈的人性，科创板集体发疯无异于婊子集体宣誓涨价，皮条客圆满完成任务，可回家烫壶好酒，剩下的只是市场的一地鸡毛，历来如此，科创板也定然逃不过，散户看看罢了，切莫当真。\n\n有人说明明知道很多却为何也亏损，以个人经验，投资有几重境界，关键要看跟自己的认知圈能否完全匹配，有人只知道公司会赚钱，却不会知道公司能否赚更多的钱，亦或是只知道公司的模式很好，但却也不知这种模式能否盈利更强的利润，总之，二级市场投资是一个踩高跷的游戏，仅知道公司赚钱是远远不够的，它考验我们的是要知道公司为何而增长以及确定公司会增长，知道增长跟知道公司会高速增长还是两回事，故，这是二级市场投资中最为艰难的，说白了，它涉及到预测未来，以及对某个行业的直观智慧，至于某些分析，看看就好，同样一句话，经历过的说出来，对于那些没有经历过的人而言，是永远也无法听懂的，能理解的也只是字面意思，还得回归那句老话，纸上得来终觉浅，绝知此事要躬行。这也是个人几十年如一日不断调研游走于底层市场的原因，看书之后你会觉得什么都懂，当你游走底层之后，你会觉得什么都不懂，很多问题都是书上没有答案的，都必须由自己提出一种不同的理论角度来解释。\n七雄圣钻继续强势不减，近期继续调研，另，录制价值套利课程中。\n\n\n\n'

In [5]:
#获取单篇文章中的图片链接
url = 'http://blog.sina.com.cn/s/blog_13122c74c0102zbud.html'
wb_data = requests.get(url)
soup = BeautifulSoup(wb_data.content)
img_link = soup.select(".articalContent.newfont_family")[0].find_all("img")[0].get("real_src")

[<div class="articalContent newfont_family" id="sina_keyword_ad_area2">
 <div>
 <p><img alt="化妆品调研随笔" real_src="http://s8.sinaimg.cn/middle/005AsbCIzy7vEfdM1M599" src="http://simg.sinajs.cn/blog7style/images/common/sg_trans.gif" title="化妆品调研随笔"/></p>
 </div>
 </div>]

In [18]:
soup.select(".articalContent.newfont_family")[0].find_all("img")[0].get("real_src")

'http://s8.sinaimg.cn/middle/005AsbCIzy7vEfdM1M599'

In [25]:
#图片下载函数
def downloadImg(img_url, file_path):
    req = requests.get(url=img_url) 
    with open(file_path, 'wb') as f: 
        f.write(req.content)
downloadImg(r'http://s8.sinaimg.cn/middle/005AsbCIzy7vEfdM1M599',r'C:\Users\dywei\python\python_excel\实例67_Python一键将喜欢的微博所有文章存为word文档\1.jpg')

以上理顺，就可以大刀阔斧地开干了。定义一个函数`to_word`，一个参数，就是上面获取到的数据字典`all_links`。设定好`header`，假装是浏览器在访问。然后新建一个word文档，设置全局字体为宋体。因为有些文章被加密，无法访问并获取内容，所以最终获取到的文章数不一定等于链接数。于是增加一个初始值为0的计数器，用于记录写入word文档中的文章数，以便心中有数。然后遍历所有文章的标题，将标题按照“1级”写入word文档，这样才能在“导航窗格”看到文章目录，方便后续选取阅读。日期和内容都作为段落写入。有些文章被加密，获取不到内容，此时`article`变量为空，所以加个if语句判断，以免程序崩溃。每写入一篇文章，计数器自动加1，然后通过`print`输出信息。最后保存文件，366页，35万字的博客就到手了，结果是美丽的！从此阅读博客文章轻松多了。

In [29]:
#写入标题，内容到word文件
import docx
from docx.oxml.ns import qn #用于应用中文字体

def to_word(all_links):
    header = {"User-Agent":"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0"}
    doc=docx.Document() #新建word文档
    doc.styles['Normal'].font.name=u'宋体'
    doc.styles['Normal']._element.rPr.rFonts.set(qn('w:eastAsia'), u'宋体')
    
    counter = 0 #计数器，用于记录写入word的文章数
    for title in all_links.keys():
        doc.add_heading(title,1)
        date = all_links[title][1][:10]#只取日期，不要时间
        doc.add_paragraph(date)
        wb_data = requests.get(all_links[title][0],headers = header)
        soup = BeautifulSoup(wb_data.content)        
        article = soup.select(".articalContent.newfont_family")
        #有些文章被加密，获取不到内容，此时article为空，所以加个if语句判断
        if article:
            text = article[0].text.replace("\xa0","")
            doc.add_paragraph(text)
            print(f"写入文章 {title} 。")
            counter += 1
    print(f"共写入 {counter} 篇文章。")
    doc.save("新浪微博文章.docx")  
    
to_word(all_links)

写入文章 投资难在慢成毁于速成 。
写入文章 投机性博弈 。
写入文章 薅羊毛操作：兖州煤业股份 。
写入文章 指数增长的重要配方：极低的边际成… 。
写入文章 价值回归套利体系目录 。
写入文章 白云机场50%获利减仓：白云山波段… 。
写入文章 企业价值是动态的：套利却依赖于静… 。
写入文章 价值套利之以自身优势构建能力圈 。
写入文章 长电继续穿越牛熊 。
写入文章 为什么要调研 。
写入文章 化妆品调研随笔 。
写入文章 科创板的疯狂：春宵一刻值千金 。
写入文章 清仓博弈仓位中烟香港：2倍套利 。
写入文章 七雄圣钻组合：依然逆势创出新高 。
写入文章 减持中烟香港：周内套利70% 。
写入文章 价值回归套利系统操作记录 。
写入文章 中烟香港马上上市：开盘直接买进 。
写入文章 清仓金力永磁获利一倍多 。
写入文章 写在海天破百之际 。
写入文章 恭喜长线账户海天到达100 。
写入文章 吴伯庸七雄圣钻组合：近五年涨幅图… 。
写入文章 清仓盈利四倍的潍柴动力 。
写入文章 价值回归系统论涪陵榨菜：从边缘到… 。
写入文章 价值回归套利系统之投资星巴克的逻… 。
写入文章 从抄底中兴谈价值回归套利系统：价… 。
写入文章 个人下周一准备建仓金力永磁 。
写入文章 为何价值回归套利系统对于估值只看… 。
写入文章 之前让大家远离康美：如今被st 。
写入文章 简谈片仔癀年报 。
写入文章 简谈让我五年十倍的价值回归系统十… 。
写入文章 吴伯庸：我的七雄圣钻组合 。
写入文章 贸易战只不过是纸老虎：它不会改变… 。
写入文章 清仓珀莱雅已持有美锦和农发种业 。
写入文章 还有最后一跌：2500点空头会被多头… 。
写入文章 短线实盘跟踪：继续持有珀莱雅到50 。
写入文章 实盘今日抄底中兴通讯：医生的医馆… 。
写入文章 我为什么说工业富联一定会破发 。
写入文章 今日已清仓智飞生物 。
写入文章 新和成：扩张下的维生素巨头 。
写入文章 继续扯扯潍柴动力的闲篇：继续持有… 。
写入文章 减持大部分白云山仓位 。
写入文章 投资是赚企业的钱还是市场的钱 。
写入文章 散户难以长拿的本质逻辑 。
写入文章 清仓信维通信：百感纠结的一场套利 。
写入文章 穿破迷雾看实质：市盈的迷局 。
写入文章 短线账户继续配置买入3倍市净的珀… 。
写入文章 华仁药业：当

![](images\result.png)